# within train set tunning

In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from godaddy_utility import *
from sklearn.model_selection import ParameterGrid,ParameterSampler
import os
from random import sample
from os.path import exists
import json
import pickle
import gc

raw = pd.read_csv('./input/raw.csv')
raw.head(5)

,row_id,cfips,county,state,first_day_of_month,microbusiness_density,active,istest,year,month,...,lastactive,lasttarget,act_lag_1,mbd_lag_2,act_lag_2,mbd_lag_3,act_lag_3,mbd_rollmea2_1,mbd_rollmea4_1,mbd_rollmea6_1
0,1001_2019-08-01,1001,Autauga County,Alabama,2019-08-01,2.856021,1249.0,0,2019,8,...,1472.0,3.286307,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1001_2019-09-01,1001,Autauga County,Alabama,2019-09-01,2.884870,1198.0,0,2019,9,...,1472.0,3.286307,-51.0,NaN,NaN,NaN,NaN,0.010101,0.010101,0.010101
2,1001_2019-10-01,1001,Autauga County,Alabama,2019-10-01,3.055843,1269.0,0,2019,10,...,1472.0,3.286307,71.0,0.010101,20.0,NaN,NaN,0.069366,0.069366,0.069366
3,1001_2019-11-01,1001,Autauga County,Alabama,2019-11-01,2.993233,1243.0,0,2019,11,...,1472.0,3.286307,-26.0,0.059265,45.0,0.010101,-6.0,0.038777,0.048878,0.048878
4,1001_2019-12-01,1001,Autauga County,Alabama,2019-12-01,2.993233,1243.0,0,2019,12,...,1472.0,3.286307,0.0,-0.020489,-26.0,0.059265,45.0,-0.020489,0.048878,0.048878


In [2]:
PRE_MOD_FOLDER = "./trainedXGB"
if os.listdir(PRE_MOD_FOLDER) == []:
    previous_version = 0
else:
    previous_version = sorted([int(s.split('_')[-1]) for s in os.listdir(PRE_MOD_FOLDER) \
                           if 'model_nof' in s])[-1]
params_version=previous_version+1

print(f"Previous version: {previous_version}")
print(f"current saving version: {params_version}")

Previous version: 0
current saving version: 1


In [3]:
#make folder is not exist
os.system(f"mkdir -p {PRE_MOD_FOLDER}/model_nof_{params_version}")
#write necessary notes: check end of the notebook
with open(f'{PRE_MOD_FOLDER}/model_nof_{params_version}/README{params_version}.txt', 'w') as f:
    f.write(f'Version {params_version}\n')

In [4]:
tunepara = True  #params_xgb
num_pset = 300
tunefeatures=True #feature_params
num_fset = 80
justretrain = False

#################################################################xgb hyperparameters
if exists(PRE_MOD_FOLDER+f"/model_nof_{previous_version}/skipping_pre_next{previous_version}"):
    with open(PRE_MOD_FOLDER+f"/model_nof_{previous_version}/skipping_pre_next{previous_version}",'rb') as fp:
        #if new feature_prebest set to False
        skipping_pre = pickle.load(fp)
else:
    skipping_pre = False

if exists(PRE_MOD_FOLDER+f"/model_nof_{previous_version}/params_xgb_best{previous_version}"):
    with open(PRE_MOD_FOLDER+f"/model_nof_{previous_version}/params_xgb_best{previous_version}", 'rb') as f: 
        params_xgb_prebest = pickle.load(f)
else:
    params_xgb_prebest = None

if exists(PRE_MOD_FOLDER+f"/model_nof_{previous_version}/params_xgb_score{previous_version}"):
    with open(PRE_MOD_FOLDER+f"/model_nof_{previous_version}/params_xgb_score{previous_version}", "rb") as fp:   
        params_xgb_prescore = pickle.load(fp)
else: 
    params_xgb_prescore = []

if skipping_pre==False:
    params_xgb_prescore = []


#################################################################feature parameterss
skipping_pre_feature = False #placeholder, depending on the new params_xgb

if exists(PRE_MOD_FOLDER+f"/model_nof_{previous_version}/feature_best{previous_version}"):
    with open(PRE_MOD_FOLDER+f"/model_nof_{previous_version}/feature_best{previous_version}", "rb") as fp:
        feature_prebest = pickle.load(fp)
else:
    feature_prebest = None

if exists(PRE_MOD_FOLDER+f"/model_nof_{previous_version}/feature_score{previous_version}"):
    with open(PRE_MOD_FOLDER+f"/model_nof_{previous_version}/feature_score{previous_version}", "rb") as fp:   
        feature_prescore = pickle.load(fp)
        pre_score = feature_prescore[-1][0]
else:
    feature_prescore = []
    pre_score = None
    


if justretrain:
    feature_prescore = []
    params_xgb_prescore = []
    
print('tune settings made')

tune settings made


In [5]:
params_xgb_prebest

In [6]:
feature_prebest

## black list

In [7]:
blacklist = [
    'North Dakota', 'Iowa', 'Kansas', 'Nebraska', 'South Dakota','New Mexico', 'Alaska', 'Vermont'
]
blacklistcfips = [
1019,1027,1029,1035,1039,1045,1049,1057,1067,1071,1077,1085,1091,1099,1101,1123,1131,1133,4001,4012,4013,4021,4023,5001,5003,5005,5017,5019,5027,5031,5035,5047,5063,5065,5071,5081,5083,5087,5091,5093,5107,5109,5115,5121,5137,5139,5141,5147,6003,6015,6027,6033,6053,6055,6057,6071,6093,6097,6103,6105,6115,8003,8007,8009,8019,8021,8023,8047,8051,8053,8055,8057,8059,8061,8065,8067,8069,8071,8073,8075,8085,8091,8093,8097,8099,8103,8105,8107,8109,8111,8115,8117,8121,9007,9009,9015,12009,12017,12019,12029,12047,12055,12065,12075,12093,12107,12127,13005,13007,13015,13017,13019,13027,13035,13047,13065,13081,13083,13099,13107,13109,13117,13119,13121,13123,13125,13127,13135,13143,13147,13161,13165,13171,13175,13181,13193,13201,13221,13225,13229,13231,13233,13245,13247,13249,13257,13279,13281,13287,13289,13293,13301,13319,15001,15005,15007,16001,16003,16005,16007,16013,16015,16017,16023,16025,16029,16031,16033,16035,16037,16043,16045,16049,16061,16063,16067,17001,17003,17007,17009,17013,17015,17023,17025,17031,17035,17045,17051,17059,17061,17063,17065,17067,17069,17075,17077,17081,17085,17087,17103,17105,17107,17109,17115,17117,17123,17127,17133,17137,17141,17143,17147,17153,17167,17169,17171,17177,17179,17181,17185,17187,17193,18001,18007,18009,18013,18015,18019,18021,18025,18035,18037,18039,18041,18053,18061,18075,18079,18083,18087,18099,18103,18111,18113,18115,18137,18139,18145,18153,18171,18179,21001,21003,21013,21017,21023,21029,21035,21037,21039,21045,21047,21055,21059,21065,21075,21077,21085,21091,21093,21097,21099,21101,21103,21115,21125,21137,21139,21141,21149,21155,21157,21161,21165,21179,21183,21191,21197,21199,21215,21217,21223,21227,21237,21239,22019,22021,22031,22039,22041,22047,22069,22085,22089,22101,22103,22109,22111,22115,22119,22121,23003,23009,23021,23027,23029,24011,24027,24029,24031,24035,24037,24039,24041,25011,25015,26003,26007,26011,26019,26021,26025,26027,26033,26037,26041,26043,26051,26053,26057,26059,26061,26065,26071,26077,26079,26083,26089,26097,26101,26103,26109,26111,26115,26117,26119,26127,26129,26131,26135,26141,26143,26155,26161,26165,27005,27011,27013,27015,27017,27021,27023,27025,27029,27047,27051,27055,27057,27065,27069,27073,27075,27077,27079,27087,27091,27095,27101,27103,27105,27107,27109,27113,27117,27119,27123,27125,27129,27131,27133,27135,27141,27147,27149,27155,27159,27167,27169,28017,28019,28023,28025,28035,28045,28049,28061,28063,28093,28097,28099,28125,28137,28139,28147,28159,29001,29015,29019,29031,29033,29041,29049,29051,29055,29057,29063,29065,29069,29075,29085,29089,29101,29103,29111,29121,29123,29125,29135,29137,29139,29143,29157,29159,29161,29167,29171,29173,29175,29177,29183,29195,29197,29199,29203,29205,29207,29209,29213,29215,29217,29223,29227,29229,30005,30009,30025,30027,30033,30035,30037,30039,30045,30049,30051,30053,30055,30057,30059,30069,30071,30073,30077,30079,30083,30085,30089,30091,30093,30101,30103,30105,30107,30109,32005,32009,32017,32023,32027,32029,32510,33005,33007,34021,34027,34033,34035,36011,36017,36023,36033,36043,36047,36049,36051,36057,36061,36067,36083,36091,36097,36103,36107,36113,36115,36121,36123,37005,37009,37011,37017,37023,37029,37031,37049,37061,37075,37095,37117,37123,37131,37137,37151,37187,37189,37197,39005,39009,39015,39017,39019,39023,39037,39039,39043,39049,39053,39057,39063,39067,39071,39077,39085,39087,39091,39097,39105,39107,39113,39117,39119,39125,39127,39129,39135,39137,39151,39153,39157,40003,40013,40015,40023,40025,40027,40035,40039,40043,40045,40053,40055,40057,40059,40065,40067,40073,40077,40079,40099,40105,40107,40111,40115,40123,40127,40129,40133,40141,40147,40151,40153,41001,41007,41013,41015,41017,41021,41025,41031,41033,41037,41051,41055,41063,41067,41069,42005,42007,42011,42013,42015,42019,42027,42029,42031,42035,42053,42057,42067,42071,42083,42085,42093,42097,42105,42111,42113,42115,42123,42125,42127,42129,44005,44007,44009,45001,45009,45021,45025,45031,45059,45067,45071,45073,45089,47001,47005,47013,47015,47019,47021,47023,47027,47035,47039,47041,47047,47055,47057,47059,47061,47069,47073,47075,47077,47083,47087,47099,47105,47121,47127,47131,47133,47135,47137,47147,47151,47153,47159,47161,47163,47169,47177,47183,47185,48001,48011,48017,48019,48045,48057,48059,48063,48065,48073,48077,48079,48081,48083,48087,48095,48101,48103,48107,48109,48115,48117,48119,48123,48125,48129,48149,48151,48153,48155,48159,48161,48165,48175,48189,48191,48195,48197,48211,48221,48229,48233,48235,48237,48239,48241,48243,48245,48255,48261,48263,48265,48267,48269,48275,48277,48283,48293,48299,48305,48311,48313,48319,48321,48323,48327,48333,48345,48347,48355,48369,48377,48379,48383,48387,48389,48401,48403,48413,48417,48431,48433,48437,48443,48447,48453,48455,48457,48461,48463,48465,48469,48471,48481,48483,48485,48487,48495,48499,49001,49009,49013,49019,49027,49031,49045,51005,51017,51025,51029,51031,51036,51037,51043,51057,51059,51065,51071,51073,51077,51079,51083,51091,51095,51097,51101,51111,51115,51119,51121,51127,51135,51147,51155,51159,51165,51167,51171,51173,51181,51183,51191,51197,51530,51590,51610,51620,51670,51678,51720,51735,51750,51770,51810,51820,53013,53019,53023,53031,53033,53037,53039,53041,53047,53065,53069,53071,53075,54013,54019,54025,54031,54033,54041,54049,54055,54057,54063,54067,54071,54077,54079,54085,54089,54103,55001,55003,55005,55007,55011,55017,55021,55025,55029,55037,55043,55047,55049,55051,55061,55065,55067,55075,55077,55091,55097,55101,55103,55109,55117,55123,55125,55127,56007,56009,56011,56015,56017,56019,56021,56027,56031,56037,56043,56045,
12061,  6095, 49025, 18073, 29029, 29097, 48419, 51830, 30067, 26095, 18159, 32001, 54065, 54027, 13043, 48177, 55069, 48137, 30087, 29007, 13055, 48295, 28157, 29037, 45061, 22053, 13199, 47171, 53001, 55041, 51195, 18127, 29151, 48307, 51009, 16047, 29133,  5145, 17175, 21027, 48357, 29179, 13023, 16077, 48371, 21057, 16039, 21143, 48435, 48317, 48475,  5129, 36041, 48075, 29017, 47175, 39167, 47109, 17189, 17173, 28009, 39027, 48133, 18129, 48217, 40081, 36021,  6005, 42099, 18051, 36055, 53051, 6109, 21073, 27019,  6051, 48055,  8083, 48503, 17021, 10003, 41061, 22001, 22011, 21205, 48223, 51103, 51047, 16069, 17033, 41011,  6035, 47145, 27083, 18165, 36055, 12001, 26159,  8125, 34017,
28141, 55119, 48405, 40029, 18125, 21135, 29073, 55115, 37149,55039, 26029, 12099, 13251, 48421, 39007, 41043, 22015, 37115,54099, 51137, 22049, 55131, 17159, 56001, 40005, 18017, 28091,47101, 27037, 29005, 13239, 21019, 55085, 48253, 51139, 40101,13283, 18049, 39163, 45049, 51113,
]

## train-valid split

In [8]:
## for train set split
ACT_THR = 1.8
ABS_THR = 1.00

raw['ypred_last'] = np.nan
raw['ypred'] = np.nan
raw['k'] = 1.
VAL = []

## Tune procedures

In [9]:
####################################################################################parameters placeholder
#https://xgboost.readthedocs.io/en/stable/parameter.html#general-parameters
params_general ={'booster': 'gbtree', 'verbosity':0, 'validate_parameters': 1}

#https://xgboost.readthedocs.io/en/stable/parameter.html#parameters-for-tree-booster
params_booster ={
    'learning_rate': 0.3,#check
    'min_split_loss': 0, #gamma. check
    'max_depth': 6,#check
    'min_child_weight': 1, #instance weight (hessian). check
    'subsample': 0.8,#check
    'colsample_bytree': 1,#check
    'reg_lambda': 1,#L2 regularization term on weights
    'reg_alpha': 0, #L1 regularization term on weights
    'max_delta_step': 0,
    'scale_pos_weight': 1,
    'tree_method': 'gpu_hist', #hist
    'predictor': 'gpu_predictor',
    'num_parallel_tree': 1
}
#https://xgboost.readthedocs.io/en/stable/parameter.html#learning-task-parameters
params_learning={
    'objective': 'reg:pseudohubererror', 'eval_metric': 'mae',
    'base_score': 0.5, 'seed': 2021
}

#https://xgboost.readthedocs.io/en/stable/parameter.html#command-line-parameters
params_train={
    'num_boost_round':500, #alias as 'n_estimators' in sklearn api
    'early_stopping_rounds':50, 'verbose_eval':False
}

params_xgb = {**params_general, **params_booster, **params_learning}

print("finish paramx_xgb initialization")

finish paramx_xgb initialization


In [10]:
##core tuning cv function
def tune_para(search_params, dtrain, skip_pre=False):
    c=0
    for psets in search_params:
        params_xgb.update(psets)
        c+=1
        if skip_pre:
            if psets in [s[2] for s in params_xgb_prescore]:
                print(f'skipping this cv batch {c}/{len(search_params)}')
                continue
        result=xgb.cv(params_xgb, dtrain, 
            num_boost_round=params_train['num_boost_round'], 
            nfold=5,
            metrics=['mae'], 
            seed=0,verbose_eval=False,as_pandas=True,
            shuffle=False,
            early_stopping_rounds=params_train['early_stopping_rounds'])
        #best score among num_boost_round
        psets_score = (round(result['test-mae-mean'].min(),7),result.shape[0], psets)
        params_xgb_prescore.append(psets_score)
        print(f"finished {c}/{len(search_params)}")
    return params_xgb_prescore

In [11]:
gridsearch_params = {
    'learning_rate': [i/100 for i in range(30,51,10)],
    'max_depth': [6,10],
    'subsample' : [i/100 for i in range(70,81,10)],
    'colsample_bytree' : [i/100 for i in range(50,81,10)],
    'min_child_weight' : [i/100 for i in range(100,201,10)],
    'reg_lambda' : [i/100 for i in range(100,301,20)] ,
    'reg_alpha' : [i/100 for i in range(40,201,20)]
}
search_params = list(ParameterSampler(param_distributions = gridsearch_params, n_iter = num_pset))
#search_params = list(ParameterGrid(param_grid = gridsearch_params))
if params_xgb_prebest in search_params:
    pass
else:
    search_params.append(params_xgb_prebest)

In [20]:
## use full train set to fit
TS = 38 # use full train set
# Build Features based in lag of target
print(build_features.__doc__)
raw, feats = build_features(raw, 'target', 'active', lags = 4)
features = ['state_i']
features += feats
print(features)

# train two same model and take average output

train_indices = (raw.istest==0) & (raw.dcount  < TS) & (raw.dcount >= 1) & (raw.lastactive>ACT_THR)  & (raw.lasttarget>ABS_THR) 
valid_indices = (raw.dcount == TS)
df = raw.loc[train_indices]
df.dropna(axis = 0, inplace= True)

dtrain=xgb.DMatrix(df[features],label=df['target'].clip(-0.0044, 0.0046),enable_categorical=True)

feature engineering
['state_i', 'mbd_lag_1', 'act_lag_1', 'mbd_lag_2', 'act_lag_2', 'mbd_lag_3', 'act_lag_3', 'mbd_rollmea2_1', 'mbd_rollmea4_1', 'mbd_rollmea6_1']


/var/folders/_0/sgv1wmb92l11hndgfl0tyvjr0000gn/T/ipykernel_78026/3042571400.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(axis = 0, inplace= True)


In [37]:
df = raw.loc[train_indices]
df[features].dropna(axis=0)

,state_i,mbd_lag_1,act_lag_1,mbd_lag_2,act_lag_2,mbd_lag_3,act_lag_3,mbd_rollmea2_1,mbd_rollmea4_1,mbd_rollmea6_1
3,0,-0.020489,-26.0,0.059265,45.0,0.010101,-6.0,0.038777,0.048878,0.048878
4,0,0.000000,0.0,-0.020489,-26.0,0.059265,45.0,-0.020489,0.048878,0.048878
5,0,-0.008066,-1.0,0.000000,-1.0,-0.020489,-27.0,-0.008066,0.030711,0.040812
6,0,-0.020129,-25.0,-0.008066,-26.0,0.000000,-26.0,-0.028195,-0.048683,0.020683
7,0,0.008217,10.0,-0.020129,-15.0,-0.008066,-16.0,-0.011912,-0.019978,0.018799
...,...,...,...,...,...,...,...,...,...,...
147331,50,0.020202,2.0,0.000000,2.0,0.010204,3.0,0.020202,0.030406,0.024865
147332,50,0.000000,0.0,0.020202,2.0,0.000000,2.0,0.020202,0.030406,0.004457
147333,50,0.000000,0.0,0.000000,0.0,0.020202,2.0,0.000000,0.020202,0.030406
147334,50,-0.009901,-1.0,0.000000,-1.0,0.000000,-1.0,-0.009901,0.010301,0.020505


In [13]:
##############################start tuning and update params_xgb, logging ,setup skipping_pre_feature
if tunepara:
    print('Start tuning')
    params_xgb_score=tune_para(search_params,dtrain,skip_pre = skipping_pre)
    del dtrain
    gc.collect()
    ##score ranking rules min: rmse,min_child_weight,learning_rate,reg_alpha
    params_xgb_score.sort(key=lambda x: (x[0],x[2]['min_child_weight'],x[2]['learning_rate'],x[2]['reg_alpha']))
    params_xgb.update(params_xgb_score[0][2])
    
    with open(f"{PRE_MOD_FOLDER}/model_nof_{params_version}/params_xgb_score{params_version}", "wb") as fp:   
        pickle.dump(params_xgb_score, fp)
    with open(f"{PRE_MOD_FOLDER}/model_nof_{params_version}/params_xgb_best{params_version}", 'wb') as f: 
        pickle.dump(params_xgb_score[0][2],f)
    if params_xgb_score[0][2] == params_xgb_prebest:
        print('best params_xgb does not change!')
        skipping_pre_feature = True
    else:
        print('best params_xgb differs!')
        skipping_pre_feature = False
        feature_prescore = []
elif params_xgb_prebest is not None:
    params_xgb.update(params_xgb_prebest)
    
    with open(f"{PRE_MOD_FOLDER}/model_nof_{params_version}/params_xgb_score{params_version}", "wb") as fp:   
        pickle.dump(params_xgb_prescore, fp)
    with open(f"{PRE_MOD_FOLDER}/model_nof_{params_version}/params_xgb_best{params_version}", 'wb') as f: 
        pickle.dump(params_xgb_prebest,f)
    skipping_pre_feature = True

Start tuning


XGBoostError: [11:48:00] /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-3.8/xgboost/src/gbm/../common/common.h:239: XGBoost version not compiled with GPU support.
Stack trace:
  [bt] (0) 1   libxgboost.dylib                    0x000000014b443c34 dmlc::LogMessageFatal::~LogMessageFatal() + 124
  [bt] (1) 2   libxgboost.dylib                    0x000000014b4e5180 xgboost::gbm::GBTree::ConfigureUpdaters() + 424
  [bt] (2) 3   libxgboost.dylib                    0x000000014b4e4d78 xgboost::gbm::GBTree::Configure(std::__1::vector<std::__1::pair<std::__1::basic_string<char, std::__1::char_traits<char>, std::__1::allocator<char> >, std::__1::basic_string<char, std::__1::char_traits<char>, std::__1::allocator<char> > >, std::__1::allocator<std::__1::pair<std::__1::basic_string<char, std::__1::char_traits<char>, std::__1::allocator<char> >, std::__1::basic_string<char, std::__1::char_traits<char>, std::__1::allocator<char> > > > > const&) + 952
  [bt] (3) 4   lib

In [ ]:

model0.fit(
    raw.loc[train_indices, features],
    raw.loc[train_indices, 'target'].clip(-0.0044, 0.0046),
)

ypred = (model0.predict(raw.loc[valid_indices, features]) + model1.predict(raw.loc[valid_indices, features]))/2
raw.loc[valid_indices, 'k'] = ypred + 1.
raw.loc[valid_indices,'k'] = raw.loc[valid_indices,'k'] * raw.loc[valid_indices,'microbusiness_density']

# Validate
lastval = raw.loc[raw.dcount==TS, ['cfips', 'microbusiness_density']].set_index('cfips').to_dict()['microbusiness_density']
dt = raw.loc[raw.dcount==TS, ['cfips', 'k']].set_index('cfips').to_dict()['k']